# Setup
This Jupyter Notebook is meant to be executed directly from Google Colab in a plug and play manner.
To enable this, the following code below can be run to set up the notebook 

In [ ]:
import os
import sys

!git clone https://github.com/terryluan12/Genrify.git
!cd Genrify && make

sys.path.insert(0, "Genrify/src")

## Preprocessing Code
The following code can be run to download the datasets and preprocess the datasets

In [1]:
from datasources import download_datasets
from preprocessing import preprocess

c:\Users\Jack\.conda\envs\aps360\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [ ]:
# Make sure to change subset_num to the subset which you are using
subset_num = None
method = None

if not os.path.isdir("Genrify/src/datasources/processed_data"):
    download_datasets("Genrify/src")

preprocess(subset_num, method, "Genrify/src")

In [2]:
from cnn import mfcc_model, training
from utils import plot
from cnn.datahandler.DataHandler import DataHandler

# Ensemble Model
This code block implements ensemle learning for all of our best models

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from ensemble import ensemble

# Ideally, we should get the dataloader for the full dataset
spec_data_dir = '/content/Genrify/src/datasources/spectrogram'
spec_data_handler = DataHandler(spec_data_dir, batch_size=32, num_workers=4)

print("Test Accuracy:", ensemble.full_model(spec_data_handler.test_loader, cuda=True))

## Example to training an model

In [1]:
# # training example
# mfcc_data_dir = os.path.join("Genrify", "src", "datasources", "mfcc")
# mfcc_data_handler = DataHandler(mfcc_data_dir, batch_size=32, num_workers=4)
# lr = 1e-4
# num_epochs = 30
# mfcc_cnn = mfcc_model.MFCC_CNN()
# print(f"Learning rate: {1e-4}")
# training.train(mfcc_cnn, mfcc_data_handler.train_loader, mfcc_data_handler.val_loader, num_epochs, lr, 32)
# model_path = training.get_model_name(mfcc_cnn.name, 32, lr, num_epochs-1)

# # plot in the notebook
# %matplotlib inline
# plot.plot_training_curve(model_path)
# plot.plot_confusion_matrix(model_path, range(10))